In [1]:
import tensorflow as tf
import os

base_dir = './train'

In [2]:

IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale = 1./255, 
    validation_split=0.2    
)
train_generator = datagen.flow_from_directory( 
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),  
    batch_size = BATCH_SIZE, 
    subset='training'
)
val_generator = datagen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 288 images belonging to 36 classes.
Found 71 images belonging to 36 classes.


In [3]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'apple': 0, 'banana': 1, 'beetroot': 2, 'bell pepper': 3, 'cabbage': 4, 'capsicum': 5, 'carrot': 6, 'cauliflower': 7, 'chilli pepper': 8, 'corn': 9, 'cucumber': 10, 'eggplant': 11, 'garlic': 12, 'ginger': 13, 'grapes': 14, 'jalepeno': 15, 'kiwi': 16, 'lemon': 17, 'lettuce': 18, 'mango': 19, 'onion': 20, 'orange': 21, 'paprika': 22, 'pear': 23, 'peas': 24, 'pineapple': 25, 'pomegranate': 26, 'potato': 27, 'raddish': 28, 'soy beans': 29, 'spinach': 30, 'sweetcorn': 31, 'sweetpotato': 32, 'tomato': 33, 'turnip': 34, 'watermelon': 35}


In [4]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
)

In [5]:
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(36, activation='softmax')
])

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/10
5/5 [==============================] - 25s 5s/step - loss: 3.7135 - accuracy: 0.0786 - val_loss: 2.7711 - val_accuracy: 0.2254
Epoch 2/10
5/5 [==============================] - 16s 3s/step - loss: 2.2674 - accuracy: 0.3272 - val_loss: 1.9164 - val_accuracy: 0.4789
Epoch 3/10
5/5 [==============================] - 16s 3s/step - loss: 1.3050 - accuracy: 0.6313 - val_loss: 1.6124 - val_accuracy: 0.5775
Epoch 4/10
5/5 [==============================] - 16s 4s/step - loss: 0.8482 - accuracy: 0.7522 - val_loss: 1.1188 - val_accuracy: 0.7183
Epoch 5/10
5/5 [==============================] - 16s 3s/step - loss: 0.4174 - accuracy: 0.8858 - val_loss: 1.0371 - val_accuracy: 0.6901
Epoch 6/10
5/5 [==============================] - 16s 3s/step - loss: 0.2633 - accuracy: 0.9570 - val_loss: 1.0125 - val_accuracy: 0.7183
Epoch 7/10
5/5 [==============================] - 16s 3s/step - loss: 0.1318 - accuracy: 0.9745 - val_loss: 0.9128 - val_accuracy: 0.7606
Epoch 8/10
5/5 [==================

In [9]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: assets
INFO:tensorflow:Assets written to: assets
